<a href="https://colab.research.google.com/github/oldaandozerskaya/age_based_classification/blob/master/feedforward_bert_age_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pymorphy2
import numpy as np
import pandas as pd
import math
from sklearn.preprocessing import OneHotEncoder
import pymorphy2, re, os, pickle

import keras
from keras import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

from keras.layers import Input, Embedding, Activation, Flatten, Dense, concatenate
from keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM
from keras.models import Model

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#get age ratings

PATH_DESCRIPTIONS = '/content/drive/My Drive/children_and_adults/corpus/description.csv'
PATH_TRAIN = '/content/drive/My Drive/children_and_adults/corpus/train/'
PATH_TEST = '/content/drive/My Drive/children_and_adults/corpus/test/'

df=pd.read_csv(PATH_DESCRIPTIONS, sep=';', header=None)

dictionary = dict(zip(df[0], df[3]))
print(dictionary)

files_train=os.listdir(PATH_TRAIN)
print('len train: {}'.format(len(files_train)))
files_test=os.listdir(PATH_TEST)
print('len test: {}'.format(len(files_test)))

In [ ]:
#get bert embs
df = pd.read_csv('/content/drive/My Drive/children_and_adults/bert_embs_train.csv', index_col=0)
bert_embs_train = df.values
df = pd.read_csv('/content/drive/My Drive/children_and_adults/bert_embs_test.csv', index_col=0)
bert_embs_test = df.values

In [ ]:
train_age_ratings = []

for f in files_train:
  train_age_ratings.append(dictionary[f])

test_age_ratings = []

for f in files_test:
  test_age_ratings.append(dictionary[f])

train_age_ratings = np.array(train_age_ratings)
train_age_ratings.shape = (4492,1)
test_age_ratings = np.array(test_age_ratings)
test_age_ratings.shape = (1000,1)

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(train_age_ratings)
train_age_ratings = enc.transform(train_age_ratings).toarray()
test_age_ratings = enc.transform(test_age_ratings).toarray()

train_age_ratings.shape

In [ ]:
#merge age ratings and bert embs

train_data = np.hstack((bert_embs_train,train_age_ratings))
test_data = np.hstack((bert_embs_test,test_age_ratings))

train_data.shape

In [ ]:
#get labels

with open('/content/drive/My Drive/children_and_adults/train_labels.pickle', 'rb') as handle:
    train_labels=pickle.load(handle)
with open('/content/drive/My Drive/children_and_adults/test_labels.pickle', 'rb') as handle:
    test_labels=pickle.load(handle)

In [ ]:
import math
border = math.ceil(0.9 * train_data.shape[0])

x_train, x_val = train_data[:border], train_data[border:]
train_labels, val_labels = train_labels[:border], train_labels[border:]

In [ ]:
#labels to categorical

train_labels = keras.utils.to_categorical(np.array(train_labels),2)
val_labels = keras.utils.to_categorical(np.array(val_labels),2)
test_labels = keras.utils.to_categorical(np.array(test_labels),2)

In [ ]:
inputs=Input(shape=(775,), name='input')
x=Dense(1024, activation='tanh', name='fully_connected_1024_tanh')(inputs)

#x=Dense(1024, activation='tanh', name='fully_connected_32')(x)
predictions=Dense(2, activation='softmax', name='output_softmax')(x)
model=Model(inputs=inputs, outputs=predictions)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

from keras.utils import plot_model
plot_model(model, to_file='fnn.png')

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

history = model.fit(x_train, train_labels, epochs=1, verbose=0, validation_data=(x_val, val_labels))
predict = np.argmax(model.predict(test_data), axis=1)
answer = np.argmax(test_labels, axis=1)

f1=f1_score(predict, answer)*100
prec=precision_score(predict, answer)*100
recall=recall_score(predict, answer)*100
accuracy=accuracy_score(predict, answer)*100

print('Готово!')
print('f1 = {}, accuracy = {}, precision = {}, recall = {}'.format(f1,accuracy,prec,recall))